In [1]:
import numpy as np
import pandas as pd
# from sklearn.preprocessing import OneHotEncoder,StandarScaler
from sklearn.metrics import accuracy_score
import random
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_auc_score

import tensorflow as tf

from collections import Counter

import math

D:\anaconda3\julianxu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_table('../data/Criteo/train.txt',header=None)

In [3]:
train.columns=['label','I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9',
       'I10', 'I11', 'I12', 'I13','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7',
       'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17',
       'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26']

In [4]:
train.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,0,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,0,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,8efede7f,3412118d,0,0,e587c466,ad3062eb,3a171ecb,3b183c5c,0,0
3,0,0.0,893,0.0,0.0,4392.0,0.0,0.0,0.0,0.0,...,1e88c74f,74ef3502,0,0,6b3a5ca6,0,3a171ecb,9117a34a,0,0
4,0,3.0,-1,0.0,0.0,2.0,0.0,3.0,0.0,0.0,...,1e88c74f,26b3c7a7,0,0,21c9516a,0,32c7478e,b34f3128,0,0


In [5]:
cross_columns = [['C1', 'C2'], ['C3', 'C4'], ['C5', 'C6'], ['C7',
       'C8'], ['C9', 'C10'], ['C11', 'C12'], ['C13', 'C14'], ['C15', 'C16'], ['C17',
       'C18'], ['C19', 'C20'], ['C21', 'C22'], ['C23', 'C24'], ['C25', 'C26']]
cont_features=['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9',
       'I10', 'I11', 'I12', 'I13']
dist_features = ['C1_C2',
       'C3_C4', 'C5_C6', 'C7_C8', 'C9_C10', 'C11_C12', 'C13_C14', 'C15_C16',
       'C17_C18', 'C19_C20', 'C21_C22', 'C23_C24', 'C25_C26','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7',
       'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17',
       'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26']

In [7]:
for cc in cross_columns:
    ccjoin = '_'.join(cc)
    train[ccjoin] = train[cc].apply(lambda x:'_'.join(x),axis=1)

In [8]:
wide_columns = ['I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9',
       'I10', 'I11', 'I12', 'I13']

In [39]:
train.to_csv('../data/Criteo/train_cross.txt',sep='\t', index=False,header=None)

In [9]:
train.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C7_C8,C9_C10,C11_C12,C13_C14,C15_C16,C17_C18,C19_C20,C21_C22,C23_C24,C25_C26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2.0,181.0,...,de7995b8_1f89b562,a73ee510_a8cd5504,b2cb9c98_37c9c164,2824a5f6_1adce6ef,8ba8b39a_891b62e7,e5ba7672_f54016b9,21ddcdc9_b1252a9d,07b5194c_0,3a171ecb_c5c50484,e8b83407_9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2.0,4.0,...,922afcc0_0b153874,a73ee510_2b53e5fb,4f1b46f3_623049e6,d7020589_b28479f6,e6c5b5cd_c92f3b61,07c540c4_b04e4670,21ddcdc9_5840adea,60f6221e_0,3a171ecb_43f13e8b,e8b83407_731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2.0,245.0,...,c78204a1_0b153874,a73ee510_3b08e48b,5f5e6091_8fe001f4,aa655a2f_07d13a8f,6dc710ed_36103458,8efede7f_3412118d,0_0,e587c466_ad3062eb,3a171ecb_3b183c5c,0_0
3,0,0.0,893,0.0,0.0,4392.0,0.0,0.0,0.0,0.0,...,2e8a689b_0b153874,a73ee510_efea433b,e51ddf94_a30567ca,3516f6e6_07d13a8f,18231224_52b8680f,1e88c74f_74ef3502,0_0,6b3a5ca6_0,3a171ecb_9117a34a,0_0
4,0,3.0,-1,0.0,0.0,2.0,0.0,3.0,0.0,0.0,...,ad9fa255_0b153874,a73ee510_5282c137,e5d8af57_66a76a26,f06c53ac_1adce6ef,8ff4b403_01adbab4,1e88c74f_26b3c7a7,0_0,21c9516a_0,32c7478e_b34f3128,0_0


In [9]:
# deep part 
freq_ = 10
# dir_feat_dict_ = 'feat_dict_' + str(freq_) + '.pkl2'
continuous_range_ = range(1, 14)
categorical_range_ = range(14, 53)
# def feat_cnt(freq_,continuous_range_,categorical_range_):
# 统计离散特征每个离散值出现的次数组成字典
deep_feat_cnt = Counter()
with open('../data/Criteo/train_cross.txt', 'r') as fin:
    for line_idx, line in enumerate(fin):
        features = line.rstrip('\n').split('\t')
        for idx in categorical_range_:
            if features[idx] == '': continue
            deep_feat_cnt.update([features[idx]])
# Only retain discrete features with high frequency
deep_dis_feat_set = set() # 高频段的离散字符
for feat, ot in deep_feat_cnt.items():
    if ot >= freq_:
        deep_dis_feat_set.add(feat)

# Create a dictionary for continuous and discrete features
deep_feat_dict = {}
tc = 1
# Continuous features
for idx in continuous_range_:
    deep_feat_dict[idx] = tc
    tc += 1 # 代表占据一列

# Discrete features
deep_cnt_feat_set = set()
with open('../data/Criteo/train_cross.txt', 'r') as fin:
    for line_idx, line in enumerate(fin):
        features = line.rstrip('\n').split('\t')
        for idx in categorical_range_:
            # 排除空字符和低频离散字符
            if features[idx] == '' or features[idx] not in deep_dis_feat_set:
                continue
            # 排除连续性数值
            if features[idx] not in deep_cnt_feat_set:
                deep_cnt_feat_set.add(features[idx])
                # 获取种类数
                deep_feat_dict[features[idx]] = tc
                tc += 1


train_label = []
train_value = []
train_idx = []

continuous_range_ = range(1, 14)
categorical_range_ = range(14, 53)
cont_max_=[]
cont_min_=[]
for cf in cont_features:
    cont_max_.append(max(train[cf]))
    cont_min_.append(min(train[cf]))
cont_diff_ = [cont_max_[i] - cont_min_[i] for i in range(len(cont_min_))]

def process_line_(line,deep_feat_dict):
    features = line.rstrip('\n').split('\t')
    feat_idx, feat_value, label = [], [], []

    # MinMax Normalization
    for idx in continuous_range_:
        if features[idx] == '':
            feat_idx.append(0)
            feat_value.append(0.0)
        else:
            feat_idx.append(deep_feat_dict[idx])
            # 归一化
            feat_value.append(round((float(features[idx]) - cont_min_[idx - 1]) / cont_diff_[idx - 1], 6))

    # 处理离散型数据
    for idx in categorical_range_:
        if features[idx] == '' or features[idx] not in deep_feat_dict:
            feat_idx.append(0)
            feat_value.append(0.0)
        else:
            feat_idx.append(deep_feat_dict[features[idx]])
            feat_value.append(1.0)
    return feat_idx, feat_value, [int(features[0])]

with open('../data/Criteo/train_cross.txt', 'r') as fin:
    for line_idx, line in enumerate(fin):
        feat_idx, feat_value, label = process_line_(line,deep_feat_dict)
        train_label.append(label)
        train_idx.append(feat_idx)
        train_value.append(feat_value)

In [16]:
f = ['0', '1.0', '1', '5.0', '0.0', '1382.0', '4.0', '15.0', '2.0', '181.0', '1.0', '2.0', '0.0', '2.0', '68fd1e64', '80e26c9b', 'fb936136', '7b4723c4', '25c83c98', '7e0ccccf', 'de7995b8', '1f89b562', 'a73ee510', 'a8cd5504', 'b2cb9c98', '37c9c164', '2824a5f6', '1adce6ef', '8ba8b39a', '891b62e7', 'e5ba7672', 'f54016b9', '21ddcdc9', 'b1252a9d', '07b5194c', '0', '3a171ecb', 'c5c50484', 'e8b83407', '9727dd16']

In [20]:
len(f)

40

In [37]:
train_label_fout.close()

In [34]:
for i in c:
     print('_'.join([f[i[0]],f[i[1]]]))

68fd1e64_80e26c9b
fb936136_7b4723c4
25c83c98_7e0ccccf
de7995b8_1f89b562
a73ee510_a8cd5504
b2cb9c98_37c9c164
2824a5f6_1adce6ef
8ba8b39a_891b62e7
e5ba7672_f54016b9
21ddcdc9_b1252a9d
07b5194c_0
3a171ecb_c5c50484
e8b83407_9727dd16


In [38]:
traincross = open('../data/Criteo/traincross.txt', 'w')
with open('../data/Criteo/train.txt', 'r') as fin:
    for line_idx, line in enumerate(fin):
        features = line.rstrip('\n').split('\t')
        # 交叉特征
        cross_range_ = [[14, 15],[16, 17],[18, 19],[20, 21],[22, 23],[24, 25],[26, 27],[28, 29],[30, 31],[32, 33],[34, 35],[36, 37],[38, 39]]
        for i in cross_range_:
            features.append('_'.join([features[i[0]],features[i[1]]]))
        string_features = ' '.join(features)
        traincross.write(string_features)
traincross.close()

In [39]:
file_path = '../data/Criteo/'
file_path +'train.txt'

'../data/Criteo/train.txt'

In [14]:
class Wide(tf.keras.layers.Layer):
    def __init__(self,units=1):
        # input_dim = num_size + embed_size = input_size
        super(Wide, self).__init__()
#         self.units = units
        self.linear = tf.keras.layers.Dense(units=units,activation='relu')
    def call(self, inputs):
        output = self.linear(inputs)
        return output

class Deep(tf.keras.layers.Layer):
    def __init__(self,num_feat,num_field,dropout_deep,deep_layer_sizes,embedding_size=10):
        # input_dim = num_size + embed_size = input_size
        super(Deep, self).__init__()
        self.num_feat = num_feat # F =features nums
        self.num_field = num_field # N =fields of a feature 
        self.dropout_deep  = dropout_deep
        
        # Embedding 这里采用embeddings层因此大小为F* M F为特征数量，M为embedding的维度
        feat_embeddings = tf.keras.layers.Embedding(num_feat, embedding_size, embeddings_initializer='uniform') # F * M 
        self.feat_embeddings = feat_embeddings
        
        # fc layer
        self.deep_layer_sizes = deep_layer_sizes
        #神经网络方面的参数
        for i in range(len(deep_layer_sizes)):
            setattr(self, 'dense_' + str(i),tf.keras.layers.Dense(deep_layer_sizes[i]))
            setattr(self, 'batchNorm_' + str(i),tf.keras.layers.BatchNormalization())
            setattr(self, 'activation_' + str(i),tf.keras.layers.Activation('relu'))
            setattr(self, 'dropout_' + str(i),tf.keras.layers.Dropout(dropout_deep[i]))
        # last layer
        self.fc = tf.keras.layers.Dense(1,activation=None,use_bias=True)
        
    def call(self,feat_index,feat_value):
        # embedding part  feat_index = inputs为输入 feat_embeddings为一个layer。
        feat_embedding_0 = self.feat_embeddings(feat_index) # Batch * N * M 
#         print(feat_value.get_shape())
        feat_embedding = tf.einsum('bnm,bn->bnm',feat_embedding_0,feat_value)

        y_deep = tf.keras.layers.Flatten()(feat_embedding)
        for i in range(len(self.deep_layer_sizes)):
            y_deep = getattr(self,'dense_' + str(i))(y_deep)
            y_deep = getattr(self,'batchNorm_' + str(i))(y_deep)
            y_deep = getattr(self,'activation_' + str(i))(y_deep)
            y_deep = getattr(self,'dropout_' + str(i))(y_deep)
        
        output = self.fc(y_deep)
        return output
    
class WideDeep(tf.keras.Model):
    def __init__(self,num_feat,num_field,dropout_deep,deep_layer_sizes,embedding_size=10):
        super().__init__()
        self.num_feat = num_feat # F =features nums
        self.num_field = num_field # N =fields of a feature 
        self.dropout_deep  = dropout_deep
        
        self.wide = Wide(units=1)
        self.deep = Deep(num_feat,num_field,dropout_deep,deep_layer_sizes)
        self.fc = tf.keras.layers.Dense(1,activation=None,use_bias=True)
        
    def call(self,num_input,feat_index,feat_value):
        x1 = self.wide(num_input)
        x2 = self.deep(feat_index,feat_value)
        
        x3 = tf.keras.layers.concatenate([x1,x2],axis=-1)
        output = self.fc(x3)
        return output

In [16]:
model = WideDeep(num_feat=len(deep_feat_dict) + 1, num_field=52, dropout_deep=[0.5, 0.5, 0.5],
                deep_layer_sizes=[400, 400],embedding_size=10)

In [17]:
train_num = train[cont_features].values

In [18]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_num,train_label,train_idx,train_value)).shuffle(10000).batch(32)

In [19]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_acc')

loss_object = tf.keras.losses.BinaryCrossentropy()

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

In [20]:
@tf.function
def train_one_step(model, optimizer, num,idx, value, label):
    with tf.GradientTape() as tape:
        output = model(num,idx,value)
        loss = loss_object(y_true=label, y_pred=output)
    grads = tape.gradient(loss, model.trainable_variables)
    grads = [tf.clip_by_norm(g, 100) for g in grads]
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label,output)

In [21]:
EPOCHS = 50
for epoch in range(EPOCHS):
    for num,label, idx, value in train_ds:
        train_one_step(model,optimizer,num,idx, value,label)
    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print (template.format(epoch+1,
                             train_loss.result(),train_accuracy.result()))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

    def call(self,feat_index,feat_value):
        # embedding part  feat_index = inputs为输入 feat_embeddings为一个layer。
        feat_embedding_0 = self.feat_embeddings(feat_index) # Batch * N * M 
#         print(feat_value.get_shape())
        feat_embedding = tf.einsum('bnm,bn->bnm',feat_embedding_0,feat_value)

        y_deep = tf.keras.layers.Flatten()(feat_embedding)
        for i in range(len(self.deep_layer_sizes)):
            y_deep = getattr(self,'dense_' + str(i))(y_deep)
            y_deep = get

Epoch 1, Loss: 10.361947059631348, Accuracy: 0.26713356375694275
Epoch 2, Loss: 10.358027458190918, Accuracy: 0.2676338255405426
Epoch 3, Loss: 10.349627494812012, Accuracy: 0.26780056953430176
Epoch 4, Loss: 10.348302841186523, Accuracy: 0.2680090069770813
Epoch 5, Loss: 10.339363098144531, Accuracy: 0.26813405752182007
Epoch 6, Loss: 10.332819938659668, Accuracy: 0.26838418841362
Epoch 7, Loss: 10.32859992980957, Accuracy: 0.2689916491508484
Epoch 8, Loss: 10.31948184967041, Accuracy: 0.26932215690612793
Epoch 9, Loss: 10.311156272888184, Accuracy: 0.270023912191391
Epoch 10, Loss: 10.302489280700684, Accuracy: 0.27053526043891907
Epoch 11, Loss: 10.29107666015625, Accuracy: 0.2714993953704834
Epoch 12, Loss: 10.281869888305664, Accuracy: 0.2725529372692108
Epoch 13, Loss: 10.27066421508789, Accuracy: 0.27379074692726135
Epoch 14, Loss: 10.261883735656738, Accuracy: 0.2751733064651489
Epoch 15, Loss: 10.25379753112793, Accuracy: 0.27697181701660156
Epoch 16, Loss: 10.243890762329102,

In [22]:
EPOCHS = 500
for epoch in range(EPOCHS):
    for num,label, idx, value in train_ds:
        train_one_step(model,optimizer,num,idx, value,label)
    template = 'Epoch {}, Loss: {}, Accuracy: {}'
    print (template.format(epoch+1,
                             train_loss.result(),train_accuracy.result()))

Epoch 1, Loss: 6.325920104980469, Accuracy: 0.5413295030593872
Epoch 2, Loss: 6.234280586242676, Accuracy: 0.5472928881645203
Epoch 3, Loss: 6.1459527015686035, Accuracy: 0.5531445145606995
Epoch 4, Loss: 6.059751510620117, Accuracy: 0.5588535070419312
Epoch 5, Loss: 5.977790832519531, Accuracy: 0.5643821954727173
Epoch 6, Loss: 5.897561073303223, Accuracy: 0.5697223544120789
Epoch 7, Loss: 5.820888042449951, Accuracy: 0.5748575925827026
Epoch 8, Loss: 5.746791362762451, Accuracy: 0.5798330307006836
Epoch 9, Loss: 5.674374103546143, Accuracy: 0.5846906304359436
Epoch 10, Loss: 5.6044440269470215, Accuracy: 0.5893779993057251
Epoch 11, Loss: 5.536790370941162, Accuracy: 0.5939527153968811
Epoch 12, Loss: 5.471333980560303, Accuracy: 0.5983878970146179
Epoch 13, Loss: 5.407954216003418, Accuracy: 0.6027140617370605
Epoch 14, Loss: 5.347165584564209, Accuracy: 0.6067877411842346
Epoch 15, Loss: 5.286743640899658, Accuracy: 0.6107207536697388
Epoch 16, Loss: 5.229099750518799, Accuracy: 0.

Epoch 129, Loss: 2.7984490394592285, Accuracy: 0.7983852028846741
Epoch 130, Loss: 2.790442705154419, Accuracy: 0.7990217208862305
Epoch 131, Loss: 2.7824296951293945, Accuracy: 0.7996512055397034
Epoch 132, Loss: 2.7745120525360107, Accuracy: 0.8002710342407227
Epoch 133, Loss: 2.7667222023010254, Accuracy: 0.8008868098258972
Epoch 134, Loss: 2.7589690685272217, Accuracy: 0.8014958500862122
Epoch 135, Loss: 2.7512989044189453, Accuracy: 0.8020983338356018
Epoch 136, Loss: 2.7437117099761963, Accuracy: 0.8026943802833557
Epoch 137, Loss: 2.7362520694732666, Accuracy: 0.8032839894294739
Epoch 138, Loss: 2.7289183139801025, Accuracy: 0.8038673400878906
Epoch 139, Loss: 2.7216155529022217, Accuracy: 0.8044445514678955
Epoch 140, Loss: 2.7142980098724365, Accuracy: 0.8050156831741333
Epoch 141, Loss: 2.707057476043701, Accuracy: 0.8055807948112488
Epoch 142, Loss: 2.699892044067383, Accuracy: 0.8061400651931763
Epoch 143, Loss: 2.6928908824920654, Accuracy: 0.8066934943199158
Epoch 144, Lo

Epoch 255, Loss: 2.196937084197998, Accuracy: 0.8457146883010864
Epoch 256, Loss: 2.1941356658935547, Accuracy: 0.8459343910217285
Epoch 257, Loss: 2.1914093494415283, Accuracy: 0.8461527228355408
Epoch 258, Loss: 2.188673257827759, Accuracy: 0.8463696241378784
Epoch 259, Loss: 2.185926914215088, Accuracy: 0.8465850949287415
Epoch 260, Loss: 2.1831698417663574, Accuracy: 0.8467991948127747
Epoch 261, Loss: 2.1804585456848145, Accuracy: 0.847011923789978
Epoch 262, Loss: 2.17773699760437, Accuracy: 0.8472232818603516
Epoch 263, Loss: 2.1751153469085693, Accuracy: 0.84743332862854
Epoch 264, Loss: 2.172483444213867, Accuracy: 0.8476419448852539
Epoch 265, Loss: 2.169895648956299, Accuracy: 0.8478493094444275
Epoch 266, Loss: 2.1672427654266357, Accuracy: 0.848055362701416
Epoch 267, Loss: 2.1646323204040527, Accuracy: 0.8482601046562195
Epoch 268, Loss: 2.1620659828186035, Accuracy: 0.8484635353088379
Epoch 269, Loss: 2.159489154815674, Accuracy: 0.848665714263916
Epoch 270, Loss: 2.1569

Epoch 381, Loss: 1.9470733404159546, Accuracy: 0.865372359752655
Epoch 382, Loss: 1.9456478357315063, Accuracy: 0.8654825091362
Epoch 383, Loss: 1.9442689418792725, Accuracy: 0.8655921220779419
Epoch 384, Loss: 1.9428966045379639, Accuracy: 0.8657012581825256
Epoch 385, Loss: 1.9414902925491333, Accuracy: 0.8658099174499512
Epoch 386, Loss: 1.940110445022583, Accuracy: 0.8659180402755737
Epoch 387, Loss: 1.9387365579605103, Accuracy: 0.8660256862640381
Epoch 388, Loss: 1.937369465827942, Accuracy: 0.8661328554153442
Epoch 389, Loss: 1.9360086917877197, Accuracy: 0.8662394881248474
Epoch 390, Loss: 1.9346531629562378, Accuracy: 0.8663456439971924
Epoch 391, Loss: 1.9332845211029053, Accuracy: 0.8664513826370239
Epoch 392, Loss: 1.9319815635681152, Accuracy: 0.8665565848350525
Epoch 393, Loss: 1.9306256771087646, Accuracy: 0.8666613101959229
Epoch 394, Loss: 1.9293347597122192, Accuracy: 0.8667656183242798
Epoch 395, Loss: 1.9280296564102173, Accuracy: 0.8668693900108337
Epoch 396, Loss:

In [42]:
a=[1,2,3,4]
b=a[:]

In [43]:
b

[1, 2, 3, 4]

In [44]:
a.append(5)

In [45]:
b

[1, 2, 3, 4]